# Trading Algorithm A
In this notebook we will go over a simplified trading algorithm which will measure return of our capital based on a certain percentage of return within a given time window. 

We have created a CSV ```changes_*``` which is a matrix: columns represent time-window size so: ```changes[24]``` represents what has been the change in percentage between 24 hours? 

From this, we can then further study different percentage distributions between those different time-windows. 

# Pseudocode
The following is a psuedo-representation of our trading algorithm

## Math notation
* $b_c$ = buying capital
* $p_n$ = prediction at timestep $n$
* $t_n$ = current timestep: 
    * $t_n$ = $p_n - n$
* $n$ = timestep, this can represent how many steps ahead in the future we will predict 
* $x_p$ = threshold percentage in float value
* $x$ = our **delta** by some percentage which is the change in price between $p_n$ and $t_n$
    * this will be represented as the **threshold** to label a **buy**
        * $x = t_{n0} + (t_{n0} * x_p)$ 
* $t_{n0}$ = opening price at $t_n$
* $p_{n0}$ = opening price at $p_n$
* $t_{ni}$ = when we enter the trade
    * $t_{ni} = t_{n0}$ when $(p_n > x)$
* $t_{nf}$ = when we get out of the trade: grabbing the last timestep price before the price dips too low
    * $t_{nf} = p_{n0 - 1}$
* ```omega_return``` = $h * t_{nf}$
* $h$ = amount of 'shares' bought
    * $h = \frac{b_c}{t_{ni}}$ 
        
### Algorithm
```
# buy and hold
if pn0 > x && bc > 0 && h == 0:
    grab opening price tn0
    buy shares with bc at tn0
    update bc to 0
    next el

# sell and wait
if pn0 < tn0 && bc == 0 && h > 0:
    grab tnf
    sell h at tnf (h * tnf)
    update bc to (h * tnf)
    update h to 0
    next el
    
else:
    pass
```

In [1]:
from omega_simulate import *

In [2]:
path = './post_processing/data_fluc/changes_1hr_USDGBP.csv'
changes_df = pd.read_csv(path)

In [3]:
changes_df.head(3)

date     open    close         1         2         3   4  \
0  2016-01-03 22:00:00  1.47352  1.47354 -0.000014       NaN       NaN NaN   
1  2016-01-03 23:00:00  1.47343  1.47356 -0.000088 -0.000075       NaN NaN   
2  2016-01-04 00:00:00  1.47364  1.47278  0.000584  0.000054  0.000068 NaN   

    5   6   7  ...  39  40  41  42  43  44  45  46  47  48  
0 NaN NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
1 NaN NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
2 NaN NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  

[3 rows x 51 columns]

In [4]:
# grabbing pn = 24
# this is the change in a 24 hour window - which is essentially a 24 hour prediction
window_24_columns = ['open', 'close', '24']
window_24 = changes_df[window_24_columns].copy()

In [5]:
window_24.head(3)

open    close  24
0  1.47352  1.47354 NaN
1  1.47343  1.47356 NaN
2  1.47364  1.47278 NaN

In [6]:
? simulate_thresh_test_trade

Signature:
 simulate_thresh_test_trade(
    sequence,
    buy_threshold,
    sell_threshold,
    start_cap,
    print_trades=True,
    return_attr=True,
)
Docstring:
This function will simulate a trade when given a sequence dataframe with a specific column holding percentage change values. 

This is an 'absolute' ideal trading simulation in which the threshold simulates what a product will produce. We are assuming we get every trade right

ARGS:
    sequence: <pandas.DataFrame> should contain: open, close, percentage change: example ['24'] represent percent change in 24 hour window.
        column 1 = open or close
        column 2 = open or close
        column 3 = change window: example: '24' = 24 hour window
    
    buy_threshold: <float> represents how much % of a change in sequence['change_window'] must there be for this to be a buy?
    
    sell_threshold: <float> represents the same as buy but for selling the asset
    
    start_cap: <int or float> represents our starting cap

In [7]:
# parameters
buying_threshold = 0.01 
selling_threshold = 0.00
starting_capital = 300000

# running simulation
simulate_thresh_test_trade(sequence=window_24, buy_threshold=buying_threshold, sell_threshold=selling_threshold, start_cap=starting_capital, print_trades=False, return_attr=False)

Starting capital: 300000
Ending capital: 608920.99599
Return on investment: 202%


In [ ]:
# parameters
buying_threshold = 0.01 
selling_threshold = 0.00
starting_capital = 300000

# running simulation
simulate_thresh_test_trade(sequence=window_24, buy_threshold=buying_threshold, sell_threshold=selling_threshold, start_cap=starting_capital, print_trades=False, return_attr=False)

In [9]:
# parameters 
x_p = 0.010 # will calculate later
bc = 300000
h = 0
n = 24
tn = 0 # initial
pn = n

start_idx = 0
end_idx = n

start_cap = bc

for i in range(len(window_24)):
    # grabbing window
    window = window_24.iloc[start_idx:end_idx]
    
    if window.iloc[-1]['24'] >= x_p and bc > 0 and h == 0:
        # grabbing opening of beginning
        pn0 = window.iloc[-1]
        tn0 = window.iloc[0]
        tni = tn0['open']
        
        # buying shares
        h = round(bc / tni)
        
#         print('----------------------------------------')
#         print('Buying')
#         print(pn0)
#         print(f'capital: {bc}')
#         print(f'bought at: {tni}')
#         print(f'bought {h} shares\n\n')
        
        # updating our buying capital to none
        bc = 0
        
    elif window.iloc[-1]['24'] <= 0.0 and bc == 0 and h > 0:
        # grabbing final buying
        tnf0 = window.iloc[-2]
        
        tnf = tnf0['open']
        
        # selling our shares
        bc = h * tnf
        
#         print('Selling')
#         print(tnf0)
#         print(f'capital: {bc}')
#         print(f'sold at: {tnf}')
#         print(f'sold {h} shares\n')
        
        # updating h
        h = 0
        
    else:
        pass 
    
    # moving window
    start_idx = start_idx + 1
    end_idx = end_idx + 1

print(f'starting capital: {start_cap}')
print(f'ending capital: {bc}')
roi = (bc / start_cap) * 100
roi = int(roi)
print(f'return on investment: {roi}%')

starting capital: 300000
ending capital: 608920.99599
return on investment: 202%


In [53]:
window_24.columns[2]

'24'

In [35]:
window_24.iloc[315+n:340+n]

open    close        24
339  1.42254  1.42134  0.002629
340  1.42136  1.42166  0.002645
341  1.42160  1.42114  0.003069
342  1.42108  1.42080  0.001649
343  1.42085  1.42244  0.001784
344  1.42244  1.42343  0.003882
345  1.42338  1.42327  0.004836
346  1.42326  1.42596  0.003610
347  1.42590  1.42820  0.007817
348  1.42826  1.42566  0.011544
349  1.42572  1.42918  0.010303
350  1.42916  1.43230  0.013416
351  1.43235  1.43288  0.014915
352  1.43283  1.43479  0.013754
353  1.43480  1.43314  0.010657
354  1.43308  1.43058  0.008707
355  1.43062  1.42949  0.006437
356  1.42955  1.43014  0.005451
357  1.43014  1.42802  0.004213
358  1.42802  1.42785  0.004177
359  1.42784  1.42640  0.003747
360  1.42675  1.42674  0.002938
361  1.42674  1.42534  0.002995
362  1.42534  1.42650  0.002814
363  1.42646  1.42838  0.003376

In [39]:
window_24.iloc[325+n-3:325+n+10]

open    close        24
346  1.42326  1.42596  0.003610
347  1.42590  1.42820  0.007817
348  1.42826  1.42566  0.011544
349  1.42572  1.42918  0.010303
350  1.42916  1.43230  0.013416
351  1.43235  1.43288  0.014915
352  1.43283  1.43479  0.013754
353  1.43480  1.43314  0.010657
354  1.43308  1.43058  0.008707
355  1.43062  1.42949  0.006437
356  1.42955  1.43014  0.005451
357  1.43014  1.42802  0.004213
358  1.42802  1.42785  0.004177

In [42]:
window_24[348:371]

open    close        24
348  1.42826  1.42566  0.011544
349  1.42572  1.42918  0.010303
350  1.42916  1.43230  0.013416
351  1.43235  1.43288  0.014915
352  1.43283  1.43479  0.013754
353  1.43480  1.43314  0.010657
354  1.43308  1.43058  0.008707
355  1.43062  1.42949  0.006437
356  1.42955  1.43014  0.005451
357  1.43014  1.42802  0.004213
358  1.42802  1.42785  0.004177
359  1.42784  1.42640  0.003747
360  1.42675  1.42674  0.002938
361  1.42674  1.42534  0.002995
362  1.42534  1.42650  0.002814
363  1.42646  1.42838  0.003376
364  1.42839  1.42996  0.005102
365  1.42996  1.43078  0.006447
366  1.43072  1.43044  0.005821
367  1.43038  1.42961  0.004883
368  1.42966  1.43050  0.004490
369  1.43044  1.42756  0.003142
370  1.42762  1.42614 -0.000406

In [ ]:
# pn = 24 at 1%

In [ ]:
# running for all windows at 1%

In [ ]:
# running for all windows at x%